# Como usar a `Embedding` layer do Keras
- Fonte: https://www.kaggle.com/rajmehra03/a-detailed-explanation-of-keras-embedding-layer

In [1]:
import keras
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, Input
from keras.models import Model

Using TensorFlow backend.


In [2]:
corp = [
    'tenis rainha preto',
    'tenis rainha branco',
    'tenis nike verde',
    'tenis nike preto',
    'tenis nike branco',
    'tenis samurai adidas verde',
    'tenis adidas preto',
    'tenis adidas branco'
]
no_docs = len(corp)

# Codificando cada palavra única
Cada palavra dos documentos acima possuem um ID único

In [3]:
vocab_size = 50 
encod_corp = []
for i, doc in enumerate(corp):
    encod_corp.append(one_hot(doc, 50))
    print("O encoding para o documento ", i + 1, " é ", one_hot(doc, 50))

O encoding para o documento  1  é  [14, 20, 25]
O encoding para o documento  2  é  [14, 20, 45]
O encoding para o documento  3  é  [14, 36, 4]
O encoding para o documento  4  é  [14, 36, 25]
O encoding para o documento  5  é  [14, 36, 45]
O encoding para o documento  6  é  [14, 17, 16, 4]
O encoding para o documento  7  é  [14, 16, 25]
O encoding para o documento  8  é  [14, 16, 45]


# Buscando a palavra com o maior cumprimento
É feito um `for` para que a frase com maior cumprimento seja selecionada, para que possamos fazer o `padding`, ou seja, deixar todas as frases com a mesma quantidade de palavras

In [4]:
import nltk
# Baixar apenas se necessário
# nltk.download('punkt')

maxlen = -1
for doc in corp:
    tokens = nltk.word_tokenize(doc)
    if(maxlen < len(tokens)):
        maxlen = len(tokens)

print("O maior número de palavras dentre todos os documentos é ", maxlen)

O maior número de palavras dentre todos os documentos é  4


In [5]:
pad_corp = pad_sequences(encod_corp, maxlen=maxlen, padding='post', value=0.0)
print("No de documentos ajustados ",len(pad_corp))

No de documentos ajustados  8


In [6]:
for i, doc in enumerate(pad_corp):
     print("O encoding após o padding do documento ", i + 1, " é ", doc)

O encoding após o padding do documento  1  é  [14 20 25  0]
O encoding após o padding do documento  2  é  [14 20 45  0]
O encoding após o padding do documento  3  é  [14 36  4  0]
O encoding após o padding do documento  4  é  [14 36 25  0]
O encoding após o padding do documento  5  é  [14 36 45  0]
O encoding após o padding do documento  6  é  [14 17 16  4]
O encoding após o padding do documento  7  é  [14 16 25  0]
O encoding após o padding do documento  8  é  [14 16 45  0]


# Criando a camada de `Embedding`
Agora todos os documentos têm o mesmo comprimento (após `padding`). E agora estamos prontos para criar e usar os embeddings.

Vou incorporar as palavras em vetores de 8 dimensões.

In [7]:
# especificando a input shape
input = Input(shape=(no_docs, maxlen), dtype='float64')

In [8]:
'''
input shape
cada documento possui 4 elementos ou palavras, valor este armazenado na variável maxlen
'''
word_input = Input(shape=(maxlen,), dtype='float64')  

# criando a camada de Embedding
word_embedding = Embedding(input_dim=vocab_size, output_dim=8, input_length=maxlen)(word_input)

word_vec = Flatten()(word_embedding) # flatten
embed_model = Model([word_input], word_vec) # unindo tudo em um model Keras

# Parâmetros da camada de `Embedding`

`input_dim` = o tamanho do vocabulário que escolheremos. Em outras palavras, é o número de palavras únicas no vocabulário.

`output_dim` = o número de dimensões que desejamos incorporar. Cada palavra será representada por um vetor desta dimensão.

`input_length` = comprimento do máximo dos documentos, que é armazenado na variável maxlen em nosso caso.

In [9]:
embed_model.compile(optimizer=keras.optimizers.Adam(lr=1e-3), loss='binary_crossentropy', metrics=['acc']) 
# compilando o model

In [10]:
print(type(word_embedding))
print(word_embedding)

<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("embedding_1/embedding_lookup/Identity_2:0", shape=(None, 4, 8), dtype=float32)


In [11]:
print(embed_model.summary()) # summary of the model

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 4)                 0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
Total params: 400
Trainable params: 400
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
embeddings=embed_model.predict(pad_corp) # finally getting the embeddings.

In [13]:
print("Shape dos Embeddings: ",embeddings.shape)
print(embeddings)

Shape dos Embeddings:  (8, 32)
[[-0.00548886 -0.03592169 -0.04380827 -0.03811025 -0.04731432 -0.00550474
  -0.02236625  0.0459923   0.04638492 -0.00064833 -0.0301376   0.0405165
   0.01600995  0.029288   -0.02455816  0.04661772 -0.00184004 -0.03448504
   0.02520813  0.0427362   0.02878145 -0.02024512 -0.02827085  0.04391212
  -0.02570124  0.03301226  0.00364808 -0.03380039  0.03013071  0.03675773
   0.00792041 -0.04911968]
 [-0.00548886 -0.03592169 -0.04380827 -0.03811025 -0.04731432 -0.00550474
  -0.02236625  0.0459923   0.04638492 -0.00064833 -0.0301376   0.0405165
   0.01600995  0.029288   -0.02455816  0.04661772 -0.04863527 -0.01664423
   0.03288651 -0.01750059  0.00012063  0.00699423 -0.02460462 -0.01855447
  -0.02570124  0.03301226  0.00364808 -0.03380039  0.03013071  0.03675773
   0.00792041 -0.04911968]
 [-0.00548886 -0.03592169 -0.04380827 -0.03811025 -0.04731432 -0.00550474
  -0.02236625  0.0459923   0.03480581  0.01236982 -0.00646875 -0.03116666
  -0.04103081  0.0468041   0.

In [14]:
embeddings=embeddings.reshape(-1,maxlen,8)
print("Shape of embeddings : ",embeddings.shape) 
print(embeddings)

Shape of embeddings :  (8, 4, 8)
[[[-0.00548886 -0.03592169 -0.04380827 -0.03811025 -0.04731432
   -0.00550474 -0.02236625  0.0459923 ]
  [ 0.04638492 -0.00064833 -0.0301376   0.0405165   0.01600995
    0.029288   -0.02455816  0.04661772]
  [-0.00184004 -0.03448504  0.02520813  0.0427362   0.02878145
   -0.02024512 -0.02827085  0.04391212]
  [-0.02570124  0.03301226  0.00364808 -0.03380039  0.03013071
    0.03675773  0.00792041 -0.04911968]]

 [[-0.00548886 -0.03592169 -0.04380827 -0.03811025 -0.04731432
   -0.00550474 -0.02236625  0.0459923 ]
  [ 0.04638492 -0.00064833 -0.0301376   0.0405165   0.01600995
    0.029288   -0.02455816  0.04661772]
  [-0.04863527 -0.01664423  0.03288651 -0.01750059  0.00012063
    0.00699423 -0.02460462 -0.01855447]
  [-0.02570124  0.03301226  0.00364808 -0.03380039  0.03013071
    0.03675773  0.00792041 -0.04911968]]

 [[-0.00548886 -0.03592169 -0.04380827 -0.03811025 -0.04731432
   -0.00550474 -0.02236625  0.0459923 ]
  [ 0.03480581  0.01236982 -0.006468

In [15]:
for i, doc in enumerate(embeddings):
    for j, word in enumerate(doc):
        print("O enconding para a palavra", j + 1, "no documento", i + 1,"é: \n\n", word)

O enconding para a palavra 1 no documento 1 é: 

 [-0.00548886 -0.03592169 -0.04380827 -0.03811025 -0.04731432 -0.00550474
 -0.02236625  0.0459923 ]
O enconding para a palavra 2 no documento 1 é: 

 [ 0.04638492 -0.00064833 -0.0301376   0.0405165   0.01600995  0.029288
 -0.02455816  0.04661772]
O enconding para a palavra 3 no documento 1 é: 

 [-0.00184004 -0.03448504  0.02520813  0.0427362   0.02878145 -0.02024512
 -0.02827085  0.04391212]
O enconding para a palavra 4 no documento 1 é: 

 [-0.02570124  0.03301226  0.00364808 -0.03380039  0.03013071  0.03675773
  0.00792041 -0.04911968]
O enconding para a palavra 1 no documento 2 é: 

 [-0.00548886 -0.03592169 -0.04380827 -0.03811025 -0.04731432 -0.00550474
 -0.02236625  0.0459923 ]
O enconding para a palavra 2 no documento 2 é: 

 [ 0.04638492 -0.00064833 -0.0301376   0.0405165   0.01600995  0.029288
 -0.02455816  0.04661772]
O enconding para a palavra 3 no documento 2 é: 

 [-0.04863527 -0.01664423  0.03288651 -0.01750059  0.00012063